In [1]:
import numpy as np
import pandas as pd

import cv2
import os
import glob
import helper_functions_07b as hf

from multiprocessing import Pool

In [2]:
DATE = ['20230302', '20230303']
SESSION = ['SM_Lek1', 'SE_Lek1']
DRONE = ['P1D1', 'P1D2', 'P2D3', 'P2D4', 'P3D5', 'P3D6']

files_directory = '/Volumes/EAS_shared/blackbuck/working/processed/Field_Recording_2023/SpatialRegistration'
video_directory = '/Volumes/EAS_shared/blackbuck/working/rawdata/Field_Recording_2023/Original/lekking'

In [ ]:
# Generate all tasks
tasks = [(date, session, drone, files_directory, video_directory) for date in DATE for session in SESSION for drone in DRONE]

# Use multiprocessing Pool to parallelize
with Pool(processes=6) as pool:  # Adjust the number of processes based on the number of tasks or on your CPU
    pool.map(hf.process_task, tasks)